<a href="https://colab.research.google.com/github/I-scream-icecream/Finance-dashboard/blob/main/predictorv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit yfinance pandas requests numpy matplotlib pandas_datareader keras scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import numpy as np
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import requests
from io import StringIO
import os

url_link = 'https://finance.yahoo.com/quote/%5ENSEI/components'
r = requests.get(url_link,headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})

# Wrap the HTML content in StringIO
html_content = StringIO(r.text)

# Read the HTML content using pandas.read_html
reader = pd.read_html(html_content)[0]

dropdown = reader.Symbol.to_list()

st.title('Dynamic Market Insights Tool v2')

ticker = st.selectbox('Pick your assets',
                          dropdown)

start = st.date_input('Start', value = pd.to_datetime('2010-01-01'))
end = st.date_input('End', value = pd.to_datetime('today'))

df = yf.download(ticker,start,end)
df = df.reset_index()

# Describing data
st.subheader(f'Data from {start} to {end}')
st.write(df.describe())

# Visualization

st.subheader(f'Closing vs Time chart for {ticker} with 50MA & 200MA')

ma50 = df['Adj Close'].rolling(50).mean()

ma200 = df['Adj Close'].rolling(200).mean()

fig1 = plt.figure(figsize = (12,6))
plt.plot(df['Date'], df['Adj Close'])
plt.plot(df['Date'], ma200, 'r', label='200-day MA')
plt.plot(df['Date'], ma50, 'y', label='50-day MA')
plt.xlabel('Date')
plt.ylabel('Adj Close')
plt.title('Close Price Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
st.pyplot(fig1)


# Button to trigger LSTM model building
if st.button('Build LSTM Model'):
    # LSTM model building code
    try:
        st.subheader(f'Creating LSTM model for {ticker}. Please wait...')
        data_training = pd.DataFrame(df['Adj Close'][0:int(len(df)*0.70)])
        data_testing = pd.DataFrame(df['Adj Close'][int(len(df)*0.70): int(len(df))])

        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler(feature_range=(0,1))

        data_training_array = scaler.fit_transform(data_training)

        model_path = '/content/keras_model_{}_{}_to_{}.h5'.format(ticker, start, end)

        if os.path.exists(model_path):
            st.write("Model already exists. Continuing...")
            # Continue with your code here
            model = load_model('keras_model_{}_{}_to_{}.h5'.format(ticker, start, end))
        else:
            # Create training sequences and labels
            x_train = []
            y_train = []

            for i in range(100, data_training_array.shape[0]):
                x_train.append(data_training_array[i - 100: i])
                y_train.append(data_training_array[i, 0])

            x_train, y_train = np.array(x_train), np.array(y_train)

            model = Sequential()
            model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
            model.add(Dropout(0.2))

            model.add(LSTM(units=60, activation='relu', return_sequences=True))
            model.add(Dropout(0.3))

            model.add(LSTM(units=80, activation='relu', return_sequences=True))
            model.add(Dropout(0.4))

            model.add(LSTM(units=120, activation='relu'))
            model.add(Dropout(0.5))

            model.add(Dense(units=1))

            model.compile(optimizer='adam', loss='mean_squared_error')
            model.fit(x_train, y_train, epochs=50)

            model.save(f'keras_model_{ticker}_{start}_to_{end}.h5')

        past_100_days = data_training.tail(100)
        final_df = past_100_days.append(data_testing, ignore_index=True)
        input_data = scaler.fit_transform(final_df)

        x_test = []
        y_test = []

        for i in range(100, input_data.shape[0]):
          x_test.append(input_data[i-100: i])
          y_test.append(input_data[i, 0])

        x_test, y_test = np.array(x_test), np.array(y_test)
        y_predicted = model.predict(x_test)

        reverser = scaler.scale_[0]
        scale_factor = 1/reverser
        y_predicted = y_predicted * scale_factor
        y_test = y_test * scale_factor

        # Display success message
        st.success("LSTM Model built successfully!")

        fig2 = plt.figure(figsize=(12,6))
        plt.plot(y_test, 'b', label = 'Original Price')
        plt.plot(y_predicted, 'r', label = 'Predicted Price')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        st.pyplot(fig2)






    except Exception as e:
        # Display error message
        st.error(f"Error occurred: {e}")

Writing app.py


In [ ]:
s! wget -q -O - ipv4.icanhazip.com

34.41.0.108


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.41.0.108:8501

npx: installed 22 in 2.639s
your url is: https://tasty-news-hear.loca.lt
2024-03-10 16:29:15.438010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 16:29:15.438093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 16:29:15.440727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 16:29:15.454223: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in